# De la visión artificial tradicional a la inteligencia artificial
* Versión 1.1 *

Para navegar hacia arriba y hacia abajo, puede usar las teclas de flecha hacia arriba y hacia abajo en su teclado <br />
Para ejecutar código en este libro de trabajo, seleccione el bloque de código y presione ** Shift + Enter ** <br />
Para editar el bloque de código, presione enter.

Los códigos de este libro de trabajo son acumulativos. (Las variables definidas siguen estando disponibles hasta que se cierra el cuaderno) <br />
¡Así que comience desde arriba y trabaje hacia abajo para evitar resultados inesperados!


Para obtener más ayuda sobre el uso de Jupyter Notebook, puede hacer clic en Ayuda> Recorrido por la interfaz de usuario en el menú de arriba, <br />
o visite https://jupyter-notebook.readthedocs.io/en/stable/ui_components.html

Experimente y pruebe sus ideas, ¡porque esa es una de las formas más rápidas de aprender!

## 1. ¿Qué pasa si no necesitamos definir reglas manualmente para resolver un problema de clasificación?

En el último taller, experimentó con varias técnicas básicas de procesamiento de imágenes y exploró cómo las computadoras podían "ver". Luego intentó que el sistema lo reconociera usando objetos que sostuvo frente a la cámara. Usted exploró una variedad de métodos, y muchos de sus métodos creativos probablemente involucraron la definición de reglas o lógica de "si-si no". Por ejemplo, reglas para lo que se consideraba colores, posición o una combinación de condiciones “autorizados”.

Pero, ¿y si no necesita definir esas reglas manualmente?

** Machine Learning ** es un subconjunto de la inteligencia artificial que se centra en la capacidad de las máquinas para aprender en función de los datos de entrenamiento. Aplicado al campo de la visión por computadora, ¿qué pasaría si pudiéramos hacer que la máquina aprenda qué es una imagen "autorizada" o "no autorizada", en lugar de tener que definir reglas para los códigos de color exactos?

En el taller de hoy, exploraremos cómo las técnicas básicas de visión por computadora se pueden combinar con el aprendizaje automático para resolver una variedad de desafíos.
1. Primero, pasaremos directamente a la creación de un modelo simple para ilustrar el aprendizaje automático.
1. Luego daremos un paso atrás para ver los pasos involucrados en la construcción de un modelo de clasificación.
1. A continuación, usamos modelos de clasificación para hacer inferencias y explorar la precisión.
1. En el camino, busque y tome nota de las limitaciones y motivaciones de los diferentes métodos y técnicas utilizados.


## Clasificación de una tarjeta en 1 de 3 categorías posibles

Echemos un vistazo rápido al desafío de las "tarjetas de acceso" nuevamente. <br />
Debajo hay 3 tarjetas (tarjetas rojas, verdes y negras) y una escena de fondo cuando no hay tarjetas colocadas frente a la cámara.
La fila superior muestra las tarjetas colocadas más lejos, mientras que la fila inferior muestra las tarjetas colocadas muy cerca de la cámara web.


<img src = "images/cards.png" style = "float: left;" />
<div style = "clear: both;"> </div>

Analicemos el problema asumiendo que las tarjetas deben mantenerse cerca de la cámara web para su validación, entonces podría ser solo una cuestión de comparar los colores de cada tarjeta (imagen) para determinar cuál de las 3 tarjetas es.

## 1.1 Extracción de funciones: selección de las funciones que se utilizarán para ayudarnos a inferir

Para este experimento, hemos decidido utilizar el color para ayudarnos a distinguir las cartas. Pero, ¿cómo seleccionaremos nuestras características de color? ¿Debemos seleccionar un punto en particular (por ejemplo, el centro de la imagen) o el color promedio de la imagen? ¿Deberíamos utilizar un canal en particular de la imagen BGR, o deberíamos convertirlo a escala de grises o cualquiera de los otros espacios de color?

La selección de nuestras características afectará la solidez de su solución, y seleccionar "características" irrelevantes no sería útil.

Por ejemplo, si intentamos utilizar el tamaño de la imagen de la cámara para determinar si se trata de una tarjeta autorizada, NO sería relevante ya que el tamaño de la imagen de la cámara no cambiará independientemente de la tarjeta que se coloque frente a la cámara.

Puede intentar experimentar con diferentes funciones. <br />
Pero mientras tanto, hagamos un experimento rápido usando el color promedio como característica:

In [1]:
import cv2
import numpy as np

# Let's read the images into memory
red_card = cv2.imread("images/cardred_close.png")
green_card = cv2.imread("images/cardgreen_close.png")
black_card = cv2.imread("images/cardblack_close.png")
background = cv2.imread("images/cardnone.png")

** Preprocesamiento y extracción de funciones **

A veces, es posible que debamos hacer un preprocesamiento en nuestros datos de entrada para asegurarnos de que tengan un formato coherente que acepte el modelo.

¿Cuáles son algunas formas en las que podemos preprocesar nuestros datos?
1. Cambiar el tamaño a un tamaño estándar.
2. Cambiar la orientación de la imagen.
3. Conversión a un espacio de color particular.

En este ejemplo en particular, nuestras imágenes de entrada cargadas ya están en un formato consistente (640x480) en el espacio de color BGR predeterminado. Pero nuestro modelo simple no usará todos los píxeles de la imagen como características para la predicción. En cambio, usaremos el color promedio como una característica para que el modelo infiera la clase a la que pertenece. Por lo tanto, a continuación definiremos un método para extraer el color promedio de cada imagen.

In [2]:
# Definir una función para extraer nuestra característica (color promedio)
def averagecolor(image):
    return np.mean(image, axis=(0, 1))

Usamos np.mean ya que el color promedio tiene 3 canales (y no un solo valor numérico). Para comprender cómo funciona np.mean, puede consultar la documentación en
https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html#numpy.mean

** Exploremos: ¿cuáles son las características extraídas (color promedio) de nuestras tarjetas roja y verde? **

In [3]:
print (averagecolor(red_card))

[ 27.35627604   4.48305664 154.21746094]


In [4]:
print (averagecolor(green_card))

[119.53976563 133.40338216  61.1089388 ]


¿Observa que los valores generados son diferentes? De hecho, sus valores están muy lejos unos de otros. ¡Esto es bueno! Esto significa que el color promedio es una buena característica para este simple problema.

** Ahora, ¿y si hubiéramos elegido usar el tamaño de la imagen como nuestra característica?

In [5]:
print (red_card.shape)

(480, 640, 3)


In [6]:
print (green_card.shape)

(480, 640, 3)


¿Sería capaz de distinguir la tarjeta roja y la tarjeta verde si solo supiera su forma?
¡No! Ya que su forma es idéntica.

¿Qué tal si supieras su colous promedio?

Como podemos ver arriba, el color promedio de la tarjeta roja y la tarjeta verde son bastante diferentes. ¡Pero el tamaño de la imagen de ambas tarjetas es exactamente el mismo! Dado que queremos usar las funciones para diferenciar las cartas, seguiremos utilizando el color promedio para ayudarnos a inferir el tipo de cartas.

Ahora crearemos variables para ingresar el valor de color promedio y la etiqueta de cada archivo de imagen. Usaremos esto más adelante para el entrenamiento de modelos. ¿Recuerda cómo se realiza este entrenamiento?

In [8]:
# Store the features (average color) and corresponding label (red/green/black/none) for classification
trainX = []
trainY = []

# loop through the cards and print the average color
for (card,label) in zip((red_card,green_card,black_card,background),("red","green","black","none")):
    print((label, averagecolor(card)))
    trainX.append(averagecolor(card))
    trainY.append(label)

('red', array([ 27.35627604,   4.48305664, 154.21746094]))
('green', array([119.53976563, 133.40338216,  61.1089388 ]))
('black', array([70.36474609, 61.85563477, 67.1775651 ]))
('none', array([247.9326888 , 241.13666016, 241.89832357]))


Recuerde del taller anterior cómo la representación de la matriz tiene el orden predeterminado [Azul, Verde, Rojo]

Observe cómo la tarjeta roja tiene un valor de rojo mucho más alto que el resto. Para la tarjeta verde, vemos que tiene valores más altos de azul y verde, y no solo verde.

trainX ahora almacena los vectores de características (características) y trainY almacena las etiquetas correspondientes.

Si se pregunta qué se almacena dentro de trainX y qué se almacena dentro de trainY, imprima las matrices y compruébelo usted mismo (comparándolas con las impresiones anteriores). Es útil que comprenda cómo se almacenan los datos en este punto.

In [9]:
print(trainX)
print(np.array(trainX).shape)      #Note how the 3 channels are stored in the array

[array([ 27.35627604,   4.48305664, 154.21746094]), array([119.53976563, 133.40338216,  61.1089388 ]), array([70.36474609, 61.85563477, 67.1775651 ]), array([247.9326888 , 241.13666016, 241.89832357])]
(4, 3)


In [10]:
print(trainY)
print(np.array(trainY).shape)

['red', 'green', 'black', 'none']
(4,)


Si toma más imágenes, puede encontrar que el color promedio no siempre es el mismo valor exacto y probablemente fluctuará debido a las condiciones de iluminación y la configuración de la cámara. Por lo tanto, entrenar un modelo generalmente implica más que unas pocas imágenes. Pero usaremos solo estas pocas imágenes solo para ilustrar el concepto.

# 1.2 Introcción del algoritmo del vecino más cercano (kNN)

Cuando colocamos una nueva tarjeta frente a la cámara, queremos determinar a cuál de estas tarjetas se parece más. En lugar de definir los códigos de color exactos, podríamos abordarlo desde el ángulo de "** ¿A cuál de nuestras tarjetas existentes conocidas es más similar a la nueva tarjeta? **"

El concepto de k-vecinos más cercanos es buscar en el conjunto de imágenes etiquetadas k imágenes más similares a la nueva imagen. Y basándose en las etiquetas de esas imágenes similares, predice la etiqueta de la nueva imagen.

Realizaremos un experimento a continuación para k = 1. Es decir, encontrar 1 imagen con el color promedio más similar al de la nueva imagen. Y use la etiqueta de esa imagen para predecir la etiqueta de la nueva imagen.

¡Analicemos cómo se hace esto!

### Primero leemos la nueva imagen en la memoria

In [11]:
new_card = cv2.imread("images/test/16.png")
new_card_features = averagecolor(new_card)

### Calcula las distancias entre las características (color promedio) de esa nueva imagen y las características de las imágenes que conocemos
Lea sobre linealg.norm [aquí] (https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.norm.html)

In [12]:
calculated_distances = []
for card in (trainX):
    calculated_distances.append(np.linalg.norm(new_card_features-card))
    
print (calculated_distances)

[117.79791641023513, 113.43645699355922, 33.497714831624535, 340.3000785919897]


### Y aquí está el resultado de a qué tarjeta es más similar:
¿Puedes adivinar con solo mirar las distancias calculadas arriba?

In [13]:
print(trainY[np.argmin(calculated_distances)])

black


Abra la subcarpeta de imágenes / prueba y verifique los colores reales de las imágenes respectivas.

Tenga en cuenta que la medida de distancia que usamos fue "np.linalg.norm ()". Puede leer más sobre esto en https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.norm.html o busque en Internet "Distancia euclidiana". En términos simples, puede tomarlo como una medida de cuán similares son los valores de matriz de (new_card_features) y (card).

Tómese un tiempo para comprender también lo que hace la última línea. Recuerde lo que está almacenado dentro de trainY en la sección 1.1.

Compruebe lo que se almacena dentro de las distancias calculadas.
¿Qué hace np.argmin?

Sugerencia: busque la documentación de numpy.argmin si es necesario.

In [14]:
print(calculated_distances)

[117.79791641023513, 113.43645699355922, 33.497714831624535, 340.3000785919897]


In [15]:
print(np.argmin(calculated_distances))

2


In [16]:
print(trainY)

['red', 'green', 'black', 'none']


In [17]:
print(trainY[np.argmin(calculated_distances)])

black


### Intentemos probar otra tarjeta
¡Recuerde revisar su carpeta para asegurarse de que el modelo realmente pueda predecir lo que queremos!

In [19]:
# Primero leemos la nueva imagen en la memoria
new_card = cv2.imread("images/test/36.png")
new_card_features = averagecolor(new_card)


# Calcule las distancias entre las características (color promedio) de esa nueva imagen frente a las características de las imágenes que conocemos
calculated_distances = []
for card in (trainX):
    calculated_distances.append(np.linalg.norm(new_card_features-card))

# Y aquí está el resultado de a qué tarjeta es más similar:
print(trainY[np.argmin(calculated_distances)])

red


### ¿Qué tal otra tarjeta?
¡Recuerde revisar su carpeta para asegurarse de que el modelo realmente pueda predecir lo que queremos!

In [20]:
# First we read the new image into memory
new_card = cv2.imread("images/test/56.png")
new_card_features = averagecolor(new_card)

# Calculate the distances between the features (average color) of that new image against the features of the images we know
calculated_distances = []
for card in (trainX):
    calculated_distances.append(np.linalg.norm(new_card_features-card))

# And here is the result of the which card it is most similar to:
print(trainY[np.argmin(calculated_distances)])

green


### Intentemos clasificar todas las tarjetas de prueba

¡Nada mal! Parece que nuestro modelo simplista ha clasificado correctamente las cartas hasta ahora.

Intentemos recorrer y clasificar todas las tarjetas en la subcarpeta de prueba.

In [23]:
from sklearn.metrics import classification_report
# Ground truth for the test images. Open the folder on your computer to see the images.
realtestY = np.array(["black","black","black","black","black",
                     "red","red","red","red","red",
                     "green","green","green","green","green",
                     "none","none","none","none","none"])
def evaluateaccuracy(filenames,predictedY):
    predictedY = np.array(predictedY)
    if (np.sum(realtestY!=predictedY)>0):
        print ("Wrong Predictions: (filename, labelled, predicted) ")
        print (np.dstack([filenames,realtestY,predictedY]).squeeze()[(realtestY!=predictedY)])
    # Calculate those predictions that match (correct), as a percentage of total predictions
    return "Correct :"+ str(np.sum(realtestY==predictedY)) + ". Wrong: "+str(np.sum(realtestY!=predictedY)) + ". Correctly Classified: " + str(np.sum(realtestY==predictedY)*100/len(predictedY))+"%"

¿Le sorprendió que no hubiera resultado para el bloque de código anterior? Eso es porque solo definimos la función para hacer la evaluación de precisión. Para obtener más información sobre las funciones en Python, puede visitar [este enlace] (https://www.datacamp.com/community/tutorials/functions-python-tutorial)

Ejecutemos el bloque de código a continuación para ver las salidas.

In [24]:
import os
path = "images/test/"
predictedY = []
filenames = []
for filename in os.listdir(path):
    img = cv2.imread(path+filename)
    img_features = averagecolor(img)
    calculated_distances = []
    for card in (trainX):
        calculated_distances.append(np.linalg.norm(img_features-card))
    prediction = trainY[np.argmin(calculated_distances)]
    
    print (filename + ": " + prediction) #Print out the inferences
    filenames.append(filename)
    predictedY.append(prediction)

# Evaluar la precisión (el paquete sklearn proporciona un informe útil)
print ()
print(classification_report(realtestY, predictedY))

# Evaluar la precisión (nuestro propio método personalizado para generar los nombres de archivo de las entradas mal clasificadas)
print ()
print (evaluateaccuracy(filenames,predictedY))


77.png: none
76.png: none
60.png: green
59.png: green
58.png: none
17.png: black
16.png: black
39.png: red
38.png: red
20.png: black
36.png: red
37.png: red
18.png: black
19.png: black
56.png: green
57.png: green
80.png: none
40.png: red
78.png: none
79.png: none

              precision    recall  f1-score   support

       black       0.00      0.00      0.00         5
       green       0.25      0.20      0.22         5
        none       0.50      0.60      0.55         5
         red       0.40      0.40      0.40         5

    accuracy                           0.30        20
   macro avg       0.29      0.30      0.29        20
weighted avg       0.29      0.30      0.29        20


Wrong Predictions: (filename, labelled, predicted) 
[['77.png' 'black' 'none']
 ['76.png' 'black' 'none']
 ['60.png' 'black' 'green']
 ['59.png' 'black' 'green']
 ['58.png' 'black' 'none']
 ['17.png' 'red' 'black']
 ['16.png' 'red' 'black']
 ['20.png' 'red' 'black']
 ['36.png' 'green' 'red']
 ['37.

** ¿Qué significa precisión y recuperación? **
¿Recuerdas que pasamos por estos durante la etapa de adquisición?

Recuerde los conceptos de verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos.

Por ejemplo, si estamos evaluando la clase roja:
- Si clasifica una imagen roja correctamente como roja, eso es un verdadero positivo.
- Si clasifica incorrectamente una imagen roja como negra, es un falso negativo.
- Si clasifica otra imagen que no es roja como roja, eso es un falso positivo.
- Si clasifica una imagen que no es roja correctamente como no roja, eso es un verdadero negativo.

La precisión es el número de verdaderos positivos dividido por (verdaderos positivos + falsos positivos), es decir, cuántos de los que se clasificaron en rojo eran en realidad rojos.

La recuperación es el número de verdaderos positivos dividido por (verdaderos positivos + falsos negativos), es decir, cuántas imágenes rojas se clasificaron correctamente en rojo cuando trató de obtener todas las imágenes rojas.

Para leer más sobre precisión y recuperación, puede buscar en Internet como de costumbre o visitar https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall

** Investiguemos la imagen mal clasificada **

Abra esa carpeta y verifique las imágenes.
Parece que 58.png se clasificó incorrectamente. ¿Por qué?

58.png

<img src = "images/test/58.png" style = "width: 400px; float: left;" />
<div style = "clear: both;"> </div>

Recuerde nuestro conjunto inicial de imágenes de entrenamiento. <br />
58.png se ve mucho más brillante que la imagen de entrenamiento para "verde", lo que puede sugerir por qué se confundió con "ninguno" (que era el "más brillante" entre las 4 imágenes de entrenamiento)

<img src = "images/cards.png" style = "float: left;" />
<div style = "clear: both;"> </div> 


Para nosotros, como humanos, es fácil para nosotros decir que 58.png debe clasificarse como verde.

Sin embargo, recuerde que la función que usamos para "entrenar" el sistema era "color promedio" y solo proporcionamos una imagen de entrenamiento.

Parece que el color promedio de 58.png está más cerca del color promedio del fondo (background.png) en lugar de la imagen de entrenamiento (cardgreen_close.png).

Se dejará como ejercicio para que calcule el color promedio de las imágenes respectivamente y descubra por qué se clasificó incorrectamente. Ese será su Desafío 1 más adelante en este cuaderno.

Mientras tanto, ¿puede pensar en una forma de mejorar el modelo?

### ¡Abre la carpeta de imágenes de prueba!
Puede abrir la carpeta de imágenes de prueba. ¿Parecen estar bajo diferentes condiciones de iluminación? Hasta ahora, solo entrenamos nuestro sistema usando un solo ejemplo para cada tarjeta de color. ¿Crees que podría ayudar tener más imágenes de entrenamiento?

## 1.3 Entrenamiento con más ejemplos
 
¿Qué tal entrenarlo con más muestras? <br />
Recuerde lo que hicimos en la sección 1.1 para obtener trainX y trainY. Si lo ha olvidado, vuelva a visitar la sección 1.1 para comprender mejor el código.

In [25]:
trainX2 = []
trainY2 = []
import os

# Let's loop through the training images in the 4 folders in the image subdirectory
path = "images/"
for label in ('red','green','black','none'):
    print ("Loading training images for the label: "+label)
    
    #Load all images inside the subfolder
    for filename in os.listdir(path+label+"/"): 
        img = cv2.imread(path+label+"/"+filename)
        img_features = averagecolor(img)
        trainX2.append(img_features)
        trainY2.append(label)

Loading training images for the label: red
Loading training images for the label: green
Loading training images for the label: black
Loading training images for the label: none


### Tarea: ¿Cuántas imágenes usamos para entrenar nuestro modelo ahora?

In [26]:
print (len(# Su código aquí))
print (len(# Su código aquí))

60
60


### Tarea: ¡Consulta con las subcarpetas!
Abra las subcarpetas rojo, verde, negro y ninguno en el directorio de imágenes de su computadora. ¿Cuántas imágenes cargamos de cada carpeta?

### Después de haber cargado más imágenes de entrenamiento, volvamos a ejecutar la prueba

In [27]:
import os
path = "images/test/"
filenames = []
predictedY = []
for filename in os.listdir(path):
    img = cv2.imread(path+filename)
    img_features = averagecolor(img)
    calculated_distances = []
    for card in (trainX2):
        calculated_distances.append(np.linalg.norm(img_features-card))
    prediction =  trainY2[np.argmin(calculated_distances)]
    
    print (filename + ": " + prediction)
    filenames.append(filename)
    predictedY.append(prediction)

# Evaluate Accuracy (the sklearn package provides a useful report)
print ()
print(classification_report(realtestY, predictedY))

# Evaluate Accuracy
print (evaluateaccuracy(filenames,predictedY))

77.png: none
76.png: none
60.png: green
59.png: green
58.png: green
17.png: black
16.png: black
39.png: red
38.png: red
20.png: black
36.png: red
37.png: red
18.png: black
19.png: black
56.png: green
57.png: green
80.png: none
40.png: red
78.png: none
79.png: none

              precision    recall  f1-score   support

       black       0.00      0.00      0.00         5
       green       0.20      0.20      0.20         5
        none       0.60      0.60      0.60         5
         red       0.40      0.40      0.40         5

    accuracy                           0.30        20
   macro avg       0.30      0.30      0.30        20
weighted avg       0.30      0.30      0.30        20

Wrong Predictions: (filename, labelled, predicted) 
[['77.png' 'black' 'none']
 ['76.png' 'black' 'none']
 ['60.png' 'black' 'green']
 ['59.png' 'black' 'green']
 ['58.png' 'black' 'green']
 ['17.png' 'red' 'black']
 ['16.png' 'red' 'black']
 ['20.png' 'red' 'black']
 ['36.png' 'green' 'red']
 ['37

** ¿Qué acabamos de hacer? **

Acabamos de ver cómo "entrenamos" el modelo para el kNN en la sección 1.1, y luego usamos el modelo para predecir a qué clase pertenecía la nueva tarjeta en la sección 1.2. Luego fuimos más allá en la sección 1.3 para explorar cómo el aumento de los datos de entrenamiento podría ayudar a mejorar la precisión, eliminando el error anterior de clasificar erróneamente "58.png" como ninguno cuando en realidad era verde.

Usamos un ejemplo muy simplificado del algoritmo kNN que encuentra los k Vecinos más cercanos para predecir la clase de la nueva imagen basada en sus vecinos más cercanos. En el ejemplo anterior, el valor de k era 1. Por lo tanto, solo buscamos el vecino más cercano (el vecino con la distancia calculada más pequeña) y predijimos el valor de la imagen de prueba según la clase del vecino más cercano.

### Tómese un momento para reflexionar

¿Cómo se compara este método con los métodos utilizados en el taller anterior?

¿Necesitó más o menos líneas de código? ¿Prefiere definir las reglas o dejar que la máquina aprenda por sí misma? Para la mayoría de ustedes, probablemente les resulte más fácil proporcionar un conjunto de imágenes de entrenamiento que tener que definir las reglas manualmente. Si le resultó más fácil definir las reglas y aún tenía un sistema bastante sólido, ¿qué técnicas utilizó?

¿Cómo podemos mejorar aún más el sistema? ¿Sería aún mejor una combinación de enfoques? ¿Funcionará esto con todo tipo de imágenes? ¿Por qué o por qué no? Escriba sus notas en la Guía de actividades del estudiante.

<br />
<video controls src="images/black_red_green.mp4" style="width:400px;" />

## 2. Pasos básicos para crear un modelo de clasificación

En la sección 1, nos lanzamos rápidamente a implementar un modelo de clasificación muy simple basado en el algoritmo kNN.
En la práctica, el entrenamiento de modelos de visión por computadora generalmente se realiza utilizando marcos como Keras, Tensorflow, Caffe y MXNet, o bibliotecas como Scikit-Learn para Python. Estos marcos y bibliotecas contienen varias herramientas y facilitan el trabajo con conjuntos de datos y algoritmos más grandes sin tener que codificar todo desde cero.

La capacitación puede llevar horas, días o incluso semanas, y a menudo requiere máquinas con GPU y capacidades informáticas más potentes. El modelo que construimos para kNN fue simplista usando matrices numpy, con el fin de ilustrar los conceptos.

Exploremos ahora los pasos que normalmente se requieren para crear un modelo de clasificación (algunos de los cuales ya se realizaron en este ejercicio):
1. Recopilación de datos
1. Preparación de datos (limpieza, etiquetado, etc.)
1. Dividir los datos en un conjunto de entrenamiento y un conjunto de prueba
1. Seleccionar un algoritmo y entrenar un modelo
1. Evaluación del desempeño

Seleccionar el algoritmo a utilizar fue solo uno de los 5 pasos. Para los algoritmos de aprendizaje automático, la preparación de datos es muy importante. Si introduce información incorrecta, el modelo resultará naturalmente incorrecto. Los datos deben ser representativos y las características utilizadas deben ser relevantes para su propósito. De lo contrario, es posible que obtenga resultados muy poco fiables.

Del mismo modo, cualquier preprocesamiento previo y las funciones que utilice para el modelo son importantes. Imagínese tratando de entrenar un modelo que reconoce flores de diferentes colores pero solo usando imágenes en escala de grises (dejando de lado las características de color importantes). Por el contrario, para el reconocimiento óptico de caracteres (OCR), el color puede no ser muy útil y puede que no se incluya en las funciones seleccionadas para el modelo.

## 3. Eso fue kNN, ¿qué tal Support Vector Machines?

Si lo piensa bien, el algoritmo k-Nemost-Neighbor realmente no aprendió mucho, básicamente almacenó los datos de entrenamiento e hizo una búsqueda cada vez que se requería una inferencia en una nueva imagen.

En tu clase de matemáticas, ¿recuerdas haber aprendido a derivar la ecuación de una línea ** y = mx + c? **

¿Qué pasaría si también pudiéramos derivar una ecuación o fórmula que pudiera usarse para predecir las diferentes clases?

## ¿Qué son los vectores de soporte?

Imagina que necesitas clasificar O de X. ¿Podrías dibujar una sola línea que separe mejor todas las X de la O?

<img src = "images/svm1.jpg" style = "width: 300px; float: left;" />
<div style = "clear: both;"> </div>

Quizás podríamos dibujar una línea (línea azul debajo). Y este es un ejemplo simple de un vector de soporte. Cualquier cosa a la izquierda / arriba de la línea podría clasificarse como X, y cualquier cosa a la derecha / abajo de la línea podría clasificarse como O.

<img src = "images/svm2.jpg" style = "width: 300px; float: left;" />
<div style = "clear: both;"> </div> 

Nota: Las matemáticas detrás de SVM estarán fuera del alcance de este taller, pero se le anima a leer más. https://www.svm-tutorial.com/2014/11/svm-understanding-math-part-1/ (En el enlace, se ilustra con diagramas cómo un solo vector lineal puede separar 2 clases distintas)

Pasemos a explorar cómo funcionan las máquinas de vectores de soporte (SVM) en la práctica, haciendo uso de la biblioteca python scikit-learn. Primero, "deriva la ecuación" del vector de soporte, luego "usa la ecuación" para ejecutar las predicciones.

### Primero se entrena el modelo

In [28]:
# Since SVM uses numerical values, we first encode our labels into numerical
from sklearn.preprocessing import LabelEncoder  #encode labels into numerical
encoder = LabelEncoder()                        #encode labels into numerical
encodedtrainY2 = encoder.fit_transform(trainY2) #encode labels into numerical

from sklearn import svm
model = svm.SVC(gamma="scale", decision_function_shape='ovr')
model.fit(trainX2, encodedtrainY2)

SVC()

¿Qué hace LabelEncoder? Veamos el resultado de la función.
Puede leer más sobre LabelEncoder [aquí] (https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621)

In [29]:
print (encodedtrainY2)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


La comprensión profunda de SVM está más allá del alcance de este módulo, pero no dude en obtener más información [aquí] (https://medium.com/all-things-ai/in-depth-parameter-tuning-for-svc- 758215394769)

Ahora, hemos obtenido nuestro modelo SVM.

### ¡Ejecutemos las predicciones!

In [30]:
import os
path = "images/test/"
filenames = []
predictedY = []
for filename in os.listdir(path):
    img = cv2.imread(path+filename)
    img_features = averagecolor(img)
    prediction = model.predict([img_features])[0]
    
    #decode the prediction
    prediction = encoder.inverse_transform([prediction])[0]
    
    print (filename + ": " + prediction)
    filenames.append(filename)
    predictedY.append(prediction)

# Evaluate Accuracy (the sklearn package provides a useful report)
print ()
print(classification_report(realtestY, predictedY))

# Evaluate Accuracy
print (evaluateaccuracy(filenames,predictedY))

77.png: none
76.png: none
60.png: green
59.png: green
58.png: green
17.png: black
16.png: black
39.png: red
38.png: red
20.png: black
36.png: red
37.png: red
18.png: black
19.png: black
56.png: green
57.png: green
80.png: none
40.png: red
78.png: none
79.png: none

              precision    recall  f1-score   support

       black       0.00      0.00      0.00         5
       green       0.20      0.20      0.20         5
        none       0.60      0.60      0.60         5
         red       0.40      0.40      0.40         5

    accuracy                           0.30        20
   macro avg       0.30      0.30      0.30        20
weighted avg       0.30      0.30      0.30        20

Wrong Predictions: (filename, labelled, predicted) 
[['77.png' 'black' 'none']
 ['76.png' 'black' 'none']
 ['60.png' 'black' 'green']
 ['59.png' 'black' 'green']
 ['58.png' 'black' 'green']
 ['17.png' 'red' 'black']
 ['16.png' 'red' 'black']
 ['20.png' 'red' 'black']
 ['36.png' 'green' 'red']
 ['37

### ¿Cuál es más preciso?

¿Crees que SVM es más eficaz para obtener clasificaciones más correctas que kNN o viceversa?

Depende del problema. Y para SVM, también hay otros parámetros que deberán ajustarse y que están fuera del alcance de este taller. Estos parámetros guiarán el proceso de generación del modelo. Por ejemplo, el modelo necesita saber qué tipo de vector de soporte generar. Una "línea recta" podría funcionar para algunos conjuntos de datos, pero para otros, podríamos necesitar una curva o vectores de soporte más complejos.

A modo de ilustración, imagínese tratando de ajustar una línea recta para clasificar las O y las X a continuación. Quizás necesite una ecuación para un círculo.

<img src = "images/svm3.jpg" style = "width: 400px; float: left;" />
<div style = "clear: both;"> </div>

Puede consultar los enlaces al final de esta sección si desea obtener más información.

Hasta este punto, entrené el modelo usando trainX2 y trainY2, luego probé nuestro modelo con un conjunto separado de imágenes y pareció funcionar bien. Sin embargo, trabajar bien en un equipo de prueba pequeño no significa que siempre funcionará bien. Probemos de nuevo con otra imagen que no se haya probado antes. El ojo humano puede distinguir fácilmente de qué color es. Pero, ¿el modelo que parece estar funcionando perfectamente hasta ahora podrá clasificarlo correctamente?

<img src = "images/cardtestagain.png" style = "width: 400px; float: left;" />
<div style = "clear: both;"> </div>

In [32]:
imagenew = cv2.imread("images/cardtestagain.png")
imagenew_features = averagecolor(imagenew)
prediction = (model.predict([imagenew_features])[0])

#decode the prediction from numerical to labels
print(encoder.inverse_transform([prediction])[0])

red


### ¿Qué salió mal?

Desafortunadamente, la imagen parece estar mal clasificada como verde en lugar de roja. <br />
Sería difícil profundizar en por qué el modelo SVM se clasificó incorrectamente en este caso sin profundizar en las matemáticas que están fuera del alcance de este taller. Una analogía simple sería que podría ser difícil intentar ajustar una curva en la ecuación de una línea recta. Así como y = mx + c sería la ecuación incorrecta para usar en una curva.

** Sugerencia adicional: ** Al diseñar soluciones con aprendizaje automático, intente entrenar el modelo más preciso, pero también tómese un tiempo para planificar las contingencias cuando el modelo puede no dar el resultado correcto. También considere cuáles podrían ser los impactos de resultados incorrectos en su aplicación y tome medidas para mitigar los riesgos. Por ejemplo, si se trata de una máquina guiada por visión por computadora, ¿existen otros sensores que también se pueden usar para activar una parada de emergencia antes de que choque contra algo?

Mientras tanto, ¿qué piensa nuestro algoritmo kNN sobre la misma imagen?

In [33]:
calculated_distances = []
for card in (trainX2):
    calculated_distances.append(np.linalg.norm(imagenew_features-card))
print(trainY2[np.argmin(calculated_distances)])

red


### ¿Eso significa que kNN siempre es más confiable?

Probemos una imagen más:

<img src = "images/cardtestagain2.png" style = "width: 400px; float: left;" />
<div style = "clear: both;"> </div>

In [34]:
imagenew = cv2.imread("images/cardtestagain2.png")
imagenew_features = averagecolor(imagenew)
calculated_distances = []
for card in (trainX2):
    calculated_distances.append(np.linalg.norm(imagenew_features-card))
    
print("SVM: "+str(encoder.inverse_transform([ model.predict([imagenew_features])[0] ])[0]))
print("kNN: "+str(trainY2[np.argmin(calculated_distances)]))


SVM: none
kNN: none


En la imagen de arriba, ¿puedes adivinar por qué kNN clasificó erróneamente el algoritmo como ninguno en lugar de verde?

Puede calcular el color promedio de la imagen para averiguar por qué.

Y sí, puede entrenar el modelo con más imágenes para mitigar estos problemas.

_Nota: Las matemáticas detrás de SVM estarán fuera del alcance de este taller, pero se le anima a leer más. https://www.svm-tutorial.com/2014/11/svm-understanding-math-part-1/ _ (En el enlace, se ilustra con diagramas cómo un solo vector lineal puede separar 2 clases distintas)

En nuestro experimento, sin embargo, lo usamos para separar más de 2 clases. Puede obtener más información sobre la clasificación de clases múltiples usando SVM y ver ejemplos de código usando la documentación en https://scikit-learn.org/stable/modules/svm.html#multi-class-classification Y recuerde buscar en Internet si necesita más ayuda.

## ¡Felicidades!

## ¡Es hora de que hagas algunas prácticas!

Ha completado una introducción muy rápida al aprendizaje automático y ha visto la progresión de un enfoque basado en reglas a un enfoque de aprendizaje automático. También ha utilizado datos de entrenamiento para entrenar los modelos kNN y SVM para la clasificación, y ha visto los resultados, así como algunas limitaciones. Ciertamente hay mucho más que aprender, ¡pero ya puedes empezar a construir!

Siempre que necesite ayuda, siempre puede buscar en su amigable Internet. Aquí hay algunos enlaces rápidos para ayudar: <br />
- https://docs.opencv.org/4.0.0/d2/d96/tutorial_py_table_of_contents_imgproc.html
- https://scikit-learn.org/stable/documentation.html

** Consejo: ** Recuerde las técnicas básicas de visión por computadora que aprendió en el taller anterior. Puede utilizarlos mientras piensa en las funciones que serían útiles para incorporar a su modelo. Espacios de color, umbralización, detección de contornos, transformaciones geométricas, manipulación directa de matrices de imágenes numerosas y más. A veces, los métodos básicos pueden ser los más efectivos para la tarea en cuestión.

Independientemente de lo que construya, tenga en cuenta su propósito y objetivo y piense en diferentes enfoques posibles. También tenga en cuenta los posibles impactos cuando un algoritmo de aprendizaje automático realiza una clasificación errónea y planifique formas de mitigar los riesgos. Por ejemplo, si sabe que su modelo identifica muy bien las tarjetas rojas, pero a veces mezcla tarjetas verdes y azules, es posible que desee diseñar una solución utilizando tarjetas rojas en lugar de verdes y azules. Y puede agregar otras capas de verificación, por ejemplo, si se detecta una tarjeta verde / negra, solicitar que el personal de seguridad realice una verificación de segunda capa.

Al igual que con otros escenarios en los que puede haber una probabilidad de error, explore la posibilidad de complementar el diseño de sus soluciones del mundo real con otras técnicas o sensores de hardware.

### Desafío 1: ¿Cuál es el color promedio de "images / test / 58.png"?

¿Recuerda que inicialmente se clasificó incorrectamente? Exploremos por qué.
Almacene el resultado en una variable "cha1"

In [35]:
import cv2
import numpy as np

image_58 = cv2.imread("images/test/58.png")
cha1 =# Su código aquí
print (cha1)

[208.77603841 223.6275293  120.75616536]


### Reto 2: ¿Cuál es el color promedio de "images / background.png"?

Almacene el resultado en una variable "cha2"

In [36]:
# Su código aquí

[247.9326888  241.13666016 241.89832357]


### Desafío 3: ¿Cuál es el color promedio de "images / cardgreen_close.png"?

Almacene el resultado en una variable "cha3"

In [37]:
# Su código aquí

[119.53976563 133.40338216  61.1089388 ]


### Desafío 4: 58.png vs Fondo. Calcula la distancia entre cha1 y cha2

Recuerde cómo se calculó la distancia euclidiana en la sección 1.2.

In [38]:
# Su código aquí

128.51161592391296

### Desafío 5: 58.png vs Green. Calcula la distancia entre cha1 y cha3

Recuerde cómo se calculó la distancia euclidiana en la sección 1.2.

In [39]:
# Su código aquí

140.2187603130577

¿Es menor la distancia en el desafío 4 (58.png frente al fondo) o la distancia en el desafío 5 (58.png frente a verde)?

Una distancia menor implica una mayor similitud. Por lo tanto, 58.png se clasificó como más similar al fondo que a la tarjeta verde en base a las 4 "imágenes de entrenamiento".

## ¡Felicidades!